In [42]:
# Pip installs for libraries
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install torch
!pip install tensorflow
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [43]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import tensorflow as tf

In [44]:
import os
path = os.path.expanduser("~/Desktop/A5-ECSE415/")

# 1 Data Preparation

In [45]:
def frames_to_video(frames_path, output_path, fps):
    # Get list of all frame files sorted by filename
    frame_files = sorted([os.path.join(frames_path, f) for f in os.listdir(frames_path) if f.endswith('.jpg')])
    # Read the first frame to get the dimensions
    first_frame = cv2.imread(frame_files[0])
    height, width, _ = first_frame.shape
    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 files
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    # Loop through and add each frame
    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        video_writer.write(frame)
    # Release video writer
    video_writer.release()
    
frames_path = path + "Tracking/Task1/images/"

# Convert frames to video
frames_to_video(frames_path, path + "task1_input.mp4", 14)

print("Video saved to", path + "task1_input.mp4")

Video saved to /Users/karimelhusseini/Desktop/A5-ECSE415/task1_input.mp4


# 2 Model Implementation

In [46]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [47]:
#Necessary imports
import sys
import glob
import time

Manually clone ultralytics yolov5

In [48]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.69 MiB | 35.62 MiB/s, done.
Resolving deltas: 100% (11718/11718), done.


In [49]:
# Creating a class for object detection which plots boxes and scores frames in addition to detecting an 
# object

class YoloDetector():

    def __init__(self):
        #Using yolov5s for our purposes of object detection, you may use a larger model
        self.model = torch.hub.load('./yolov5', 'yolov5s', pretrained = True, source='local')
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('Using Device: ', self.device)
    
    def score_frame(self, frame):
        self.model.to(self.device)
        downscale_factor = 2
        width = int(frame.shape[1] / downscale_factor)
        height = int(frame.shape[0] / downscale_factor)
        frame = cv2.resize(frame, (width, height))

        results = self.model(frame)

        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

        return labels, cord
    
    def class_to_label(self, x):
        return self.classes[int(x)]
    
    def plot_boxes(self, results, frame, height, width, confidence=0.3):

        labels, cord = results
        detections = []

        n = len(labels)
        x_shape, y_shape = width, height

        for i in range(n):
            row = cord[i]

            if row[4]>=confidence:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                
                if self.class_to_label(labels[i]) == 'person':

                    x_center = x1 + (x2-x1)
                    y_center = y1 + ((y2-y1) / 2)

                    tlwh = np.asarray([x1, y1, int(x2-x1), int(y2-y1)], dtype = np.float32)
                    confidence = float(row[4].item())
                    feature = 'person'

                    detections.append(([x1, y1, int(x2-x1), int(y2-y1)], row[4].item(), 'person'))
        
        return frame, detections

In [50]:
cap = cv2.VideoCapture(path + "task1_input.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(path+'task1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))


#Initializing the detection class
detector = YoloDetector()

YOLOv5 🚀 v7.0-388-g882c35fc Python-3.12.0 torch-2.3.1 CPU

ERROR: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
Re-attempting https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt to yolov5s.pt...
######################################################################## 100.0%########################                                   55.1%

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Using Device:  cpu


In [51]:
!pip install deep-sort-realtime

Defaulting to user installation because normal site-packages is not writeable
    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [52]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [53]:
from deep_sort_realtime.deepsort_tracker import DeepSort

#Initialise the object tracker class
object_tracker = DeepSort()

In [54]:
tracking_results_file = open('tracking_results.txt', 'w')
frame_count = 1

while cap.isOpened():
    success, img = cap.read()
    
    if success or img is not None:

        start = time.perf_counter()

        results = detector.score_frame(img)
        img,detections = detector.plot_boxes(results, img, height=img.shape[0], width=img.shape[1], confidence=0.1)

        tracks = object_tracker.update_tracks(detections, frame=img) 
        # NOTE: Bounding box expects to be a list of detections, each in tuples of ([left, top, w, h], confidence, detection class)
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            ltrb = track.to_ltrb()

            bbox = ltrb
            bb_left = int(bbox[0])
            bb_top = int(bbox[1])
            bb_width = int(bbox[2] - bbox[0])
            bb_height = int(bbox[3] - bbox[1])
            
            # Write tracking results to the text file
            tracking_results_file.write(f"{frame_count}, {track_id}, {bb_left}, {bb_top}, {bb_width}, {bb_height}\n")

            cv2.rectangle(img, (int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3])),(0,0,255),2)
            cv2.putText(img, "ID: " + str(track_id), (int(bbox[0]),int(bbox[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        end = time.perf_counter()
        totalTime = end-start
        fps = 1/totalTime

        cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.imshow('img', img)
        
        # Write the frame to the output video
        out.write(img)
        
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        print('invlid frame or video ended')
        break

tracking_results_file.close()

cap.release()
out.release()

# Destroy all windows
cv2.destroyAllWindows()
cv2.waitKey(1)

print("Video saved to", path + "task1.mp4")

invlid frame or video ended
Video saved to /Users/karimelhusseini/Desktop/A5-ECSE415/task1.mp4


# 3 Model Evaluation

In [55]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def read_data(file_path, has_confidence=False):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            frame = int(parts[0])
            while len(data) <= frame:
                data.append([])
            id = int(parts[1])
            bb_left = int(parts[2])
            bb_top = int(parts[3])
            bb_width = int(parts[4])
            bb_height = int(parts[5])
            bbox = (bb_left, bb_top, bb_left + bb_width, bb_top + bb_height, id)
            data[frame].append(bbox)
    return data

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def mota_score(ground_truths, predictions):
    total_gt = sum(len(gt) for gt in ground_truths)
    total_false_negatives = 0
    total_false_positives = 0
    total_id_switches = 0
    prev_frame_assignments = {}

    for frame_index in range(min(len(ground_truths)-1, len(predictions)-1)):
        gt_boxes = ground_truths[frame_index]
        pred_boxes = predictions[frame_index]
        iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)))

        for i, gt in enumerate(gt_boxes):
            for j, pred in enumerate(pred_boxes):
                iou_matrix[i, j] = iou(gt, pred)

        row_ind, col_ind = linear_sum_assignment(-iou_matrix)
        matched_indices = set(zip(row_ind, col_ind))
        frame_false_negatives = len(gt_boxes) - len(row_ind)
        frame_false_positives = len(pred_boxes) - len(col_ind)
        
         # **Print frames with false positives**
        if frame_false_positives > 0:
            print(f"False Positives Detected at Frame {frame_index}")

            # Optionally, list the false positive bounding boxes
            matched_preds = set(col_ind)
            for idx, pred in enumerate(pred_boxes):
                if idx not in matched_preds:
                    print(f"\tFalse Positive Bounding Box: {pred[:4]}, ID: {pred[4]}")

        frame_id_switches = 0
        current_frame_assignments = {}
        for r, c in matched_indices:
            if iou_matrix[r, c] >= 0.5:  # Consider matching if IoU is above 0.5
                current_id = pred_boxes[c][4]
                gt_id = gt_boxes[r][4]
                if gt_id in prev_frame_assignments and prev_frame_assignments[gt_id] != current_id:
                    print(f"Identity Switch Detected at Frame {frame_index}: GT ID {gt_id} changed from {prev_frame_assignments[gt_id]} to {current_id}")
                    frame_id_switches += 1
                current_frame_assignments[gt_id] = current_id


        total_false_negatives += frame_false_negatives
        total_false_positives += frame_false_positives
        total_id_switches += frame_id_switches
        
        prev_frame_assignments.update(current_frame_assignments)
    
    results_summary = {
        'total_gt': total_gt,
        'total_false_negatives': total_false_negatives,
        'total_false_positives': total_false_positives,
        'total_id_switches': total_id_switches
    }

    mota = 1 - (total_false_negatives + total_false_positives + total_id_switches) / total_gt
    return mota, results_summary



In [56]:
# Example usage:
ground_truths = read_data(path+'Tracking/Task1/gt/gt.txt', has_confidence=True)
tracking_results = read_data(path+'tracking_results.txt')

mota_score_result = mota_score(ground_truths, tracking_results)

print("MOTA Score:", mota_score_result[0])
print("Results Summary:", mota_score_result[1])

Identity Switch Detected at Frame 205: GT ID 4 changed from 7 to 24
Identity Switch Detected at Frame 231: GT ID 5 changed from 9 to 28
False Positives Detected at Frame 271
	False Positive Bounding Box: (-177, 124, -62, 398), ID: 29
False Positives Detected at Frame 272
	False Positive Bounding Box: (-185, 123, -68, 402), ID: 29
False Positives Detected at Frame 273
	False Positive Bounding Box: (-192, 121, -73, 404), ID: 29
False Positives Detected at Frame 274
	False Positive Bounding Box: (-200, 119, -79, 406), ID: 29
False Positives Detected at Frame 275
	False Positive Bounding Box: (-207, 118, -84, 410), ID: 29
False Positives Detected at Frame 276
	False Positive Bounding Box: (-215, 116, -91, 412), ID: 29
False Positives Detected at Frame 277
	False Positive Bounding Box: (-222, 114, -96, 414), ID: 29
Identity Switch Detected at Frame 317: GT ID 5 changed from 28 to 36
MOTA Score: 0.33982114676486064
Results Summary: {'total_gt': 1901, 'total_false_negatives': 1245, 'total_fal

# 4 Pedestrian Counting on Kaggle

In [57]:
frames_path = path + "Tracking/Task2/images/"

# Convert frames to video
frames_to_video(frames_path, path + "task2_input.mp4", 14)

print("Video saved to", path + "task2_input.mp4")

Video saved to /Users/karimelhusseini/Desktop/A5-ECSE415/task2_input.mp4


In [58]:
cap = cv2.VideoCapture(path + "task2_input.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(path+'task2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

In [59]:
tracking_results_file = open('tracking_results_task2.txt', 'w')
frame_count = 1

while cap.isOpened():
    success, img = cap.read()
    
    if success or img is not None:

        start = time.perf_counter()

        results = detector.score_frame(img)
        img,detections = detector.plot_boxes(results, img, height=img.shape[0], width=img.shape[1], confidence=0.1)

        tracks = object_tracker.update_tracks(detections, frame=img) 
        # NOTE: Bounding box expects to be a list of detections, each in tuples of ([left, top, w, h], confidence, detection class)
        
        if len(tracks) == 0:
            print('No tracks detected for frame', frame_count)
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            ltrb = track.to_ltrb()

            bbox = ltrb
            bb_left = int(bbox[0])
            bb_top = int(bbox[1])
            bb_width = int(bbox[2] - bbox[0])
            bb_height = int(bbox[3] - bbox[1])
            
            # Write tracking results to the text file
            tracking_results_file.write(f"{frame_count}, {track_id}, {bb_left}, {bb_top}, {bb_width}, {bb_height}\n")

            cv2.rectangle(img, (int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3])),(0,0,255),2)
            cv2.putText(img, "ID: " + str(track_id), (int(bbox[0]),int(bbox[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        end = time.perf_counter()
        totalTime = end-start
        fps = 1/totalTime

        cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        cv2.imshow('img', img)
        
        # Write the frame to the output video
        out.write(img)
        
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        print('invlid frame or video ended')
        break

tracking_results_file.close()

cap.release()
out.release()

# Destroy all windows
cv2.destroyAllWindows()
cv2.waitKey(1)

print("Video saved to", path + "task2.mp4")

invlid frame or video ended
Video saved to /Users/karimelhusseini/Desktop/A5-ECSE415/task2.mp4


### Counting the number of boxes in each frame, to estimate the number of pedestrians per frame

In [62]:
import csv

def count_bboxes_per_frame(data):
    bbox_counts = {}
    for frame_index, bboxes in enumerate(data):
        # Count the number of bounding boxes in the current frame
        if frame_index != 0:
            bbox_count = len(bboxes)
            bbox_counts[frame_index] = bbox_count
    return bbox_counts

task2_tracking_results = read_data(path+'tracking_results_task2.txt')
pedestrian_counts = count_bboxes_per_frame(task2_tracking_results)

with open('task2_count.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', 'Count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in pedestrian_counts.items():
        
        writer.writerow({'Number': key, 'Count': value})
        
print("Pedestrian counts saved to task2_count.csv")

Pedestrian counts saved to task2_count.csv
